In [124]:
from gurobipy import GRB, quicksum
import gurobipy as gb

In [125]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [126]:
df = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/updated_gym_data.csv')

In [127]:
df.head(10)

,Exercise,Category,BodyPart,Equipment,Difficulty,Stimulus-to-Fatigue,Expected Time,Hypertrophy Rating
0,Bench Press With Short Bands,Powerlifting,Chest,Bands,Beginner,0.817884,15.518089,0.596124
1,Hip Lift with Band,Powerlifting,Glutes,Bands,Beginner,0.768902,14.655351,0.623237
2,Band Good Morning (Pull Through),Powerlifting,Hamstrings,Bands,Beginner,0.792188,16.292358,0.601159
3,Speed Box Squat,Powerlifting,Quadriceps,Bands,Intermediate,0.599044,17.109781,0.800347
4,Partner plank band row,Strength,Abdominals,Bands,Intermediate,0.730726,14.212727,0.461565
5,Banded crunch isometric hold,Strength,Abdominals,Bands,Intermediate,0.716055,12.619634,0.416539
6,FYR Banded Plank Jack,Strength,Abdominals,Bands,Intermediate,0.766785,12.604063,0.534897
7,Banded crunch,Strength,Abdominals,Bands,Intermediate,0.765022,12.996750,0.464355
8,Crunch,Strength,Abdominals,Bands,Intermediate,0.710541,14.824696,0.494665
9,Decline band press sit-up,Strength,Abdominals,Bands,Intermediate,0.764133,13.953021,0.437804


In [128]:
df.columns

Index(['Exercise', 'Category', 'BodyPart', 'Equipment', 'Difficulty',
       'Stimulus-to-Fatigue', 'Expected Time', 'Hypertrophy Rating'],
      dtype='object')

In [129]:
df['BodyPart'].unique()

array(['Chest', 'Glutes', 'Hamstrings', 'Quadriceps', 'Abdominals',
       'Adductors', 'Abductors', 'Biceps', 'Calves', 'Forearms', 'Lats',
       'Lower Back', 'Middle Back', 'Traps', 'Shoulders', 'Triceps',
       'Neck'], dtype=object)

In [130]:
# getting data from data set
exercise_ids = df.index
hypertrophy_ratings = df['Hypertrophy Rating']
sfr_values = df['Stimulus-to-Fatigue']
body_parts = df['BodyPart']
categories = df['Category']
equipment_types = df['Equipment']
Difficulty_rating = df['Difficulty']

### 2. c) Using Gurobi, what is the optimal hypertrophy rating using all constraints? 

In [131]:
# Create a new optimization model
model = gb.Model("Hypertrophy Optimization")

# Decision variables
num_exercises = len(df)
x = model.addVars(num_exercises, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="ExerciseProportion")

# Objective function: Maximizes the total hypertrophy rating across all exercises based on their proportions.
model.setObjective(
        quicksum(df.loc[i, 'Hypertrophy Rating'] * x[i] for i in range(num_exercises)),
        GRB.MAXIMIZE
    )

# Adding the constraints
## 1. Limit the proportion per exercise to a maximum of 5%.
for i in range(num_exercises):
    model.addConstr(x[i] <= 0.05, name=f"MaxProportion_{i}")

# 2. Enforce minimum allocations for specific and general body parts.
general_min_allocation = 0.025  # 2.5%
specific_min_allocations = {
    'Traps': 0.005,      # 0.5%
    'Neck': 0.005,       # 0.5%
    'Forearms': 0.005,   # 0.5%
    'Abdominals': 0.04   # 4%
}

# General and specific allocation constraints for body parts
unique_body_parts = df['BodyPart'].unique()
for part in unique_body_parts:
    if part in specific_min_allocations:
        model.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= specific_min_allocations[part],
            f"{part}Min"
        )
    else:
        model.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= general_min_allocation,
            f"{part}GeneralMin"
        )

# 3. Ensure leg muscles allocation is at least 2.6 times the upper body allocation.
leg_muscles = ['Adductors', 'Abductors', 'Calves', 'Glutes', 'Hamstrings', 'Quadriceps']
upper_body_muscles = ['Chest', 'Lower Back', 'Middle Back', 'Biceps', 'Traps', 'Triceps', 
                      'Shoulders', 'Abdominals', 'Forearms', 'Neck', 'Lats']

leg_upper_body_ratio_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in leg_muscles) >= 
    2.6 * gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in upper_body_muscles),
    "LegUpperBodyRatio"
)

# 4. Balance biceps and triceps allocations with chest, lower back, and middle back allocations.
muscle_group_balance_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Biceps', 'Triceps']) ==
    gb.quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Chest', 'Lower Back', 'Middle Back']),
    "MuscleGroupBalance"
)

# 5. Restrict the overall Stimulus-to-Fatigue Ratio (SFR) to a maximum of 0.55
sfr_constraint = model.addConstr(
    gb.quicksum(sfr_values[i] * x[i] for i in exercise_ids) <= 0.55,
    "SFRConstraint"
)

# 6. Maintain beginner ≥ 1.4 × intermediate ≥ advanced difficulty ratios.
beginner_intermediate_ratio_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Beginner') >= 
    1.4 * gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Intermediate'),
    "BeginnerIntermediateRatio"
)

intermediate_advanced_ratio_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Intermediate') >= 
    1.1 * gb.quicksum(x[i] for i in exercise_ids if Difficulty_rating[i] == 'Advanced'),
    "IntermediateAdvancedRatio"
)

# 7. Set minimum and maximum allocations for Strongman, Powerlifting, and Olympic Weightlifting exercises.
# Strongman exercises ≤ 8%
strongman_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Strongman') <= 0.08,
    "StrongmanMax"
)

# Powerlifting exercises ≥ 9%
powerlifting_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Powerlifting') >= 0.09,
    "PowerliftingMin"
)

# Olympic Weightlifting exercises ≥ 10%
olympic_weightlifting_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if categories[i] == 'Olympic Weightlifting') >= 0.10,
    "OlympicWeightliftingMin"
)

# 8. Ensure at least 60% of exercises involve essential equipment types.
essential_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
equipment_constraint = model.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if equipment_types[i] in essential_equipment) >= 0.6,
    "EssentialEquipmentMin"
)

# Ensure total proportion is 1
model.addConstr(gb.quicksum(x[i] for i in range(len(df))) == 1, "Total_Proportion")

# Solve the optimization model to find the optimal proportions for each exercise while adhering to all constraints.
# The model successfully converges to an optimal solution with an objective value of 0.7672, maximizing hypertrophy.
model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model.computeIIS()
    model.write("infeasible_constraints.ilp")
    
# Check the optimization status
if model.status == GRB.OPTIMAL:
    print("The optimal solution: ", model.objVal)
    # Optionally, display the decision variables
    for i in exercise_ids:
        if x[i].x > 0:
            print(f"Exercise {df.loc[i, 'Exercise']}: Proportion = {x[i].x:.4f}")
elif model.status == GRB.INFEASIBLE:
    print("The model is infeasible.")
elif model.status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print("Optimization was stopped with status", model.status)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2664 rows, 2637 columns and 20237 nonzeros
Model fingerprint: 0x25a0cea5
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 2638 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 2637 columns, 15326 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.805000e+00   0.000000e+00      0s
      31    7.6726674e-01   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.00 seconds (0.01 work units)
Optimal objective  7.672667395e-01
The optimal solution:  0.7672667395203319
Exercise Clean Deadlift: Proportion = 0.0500
Exercise Muscle Snatch: Proportion = 0.0500
Exercise Clean Shrug: Proportion 

In [132]:
from gurobipy import Model, GRB, quicksum

# Initialize the Gurobi model
model = Model("Hypertrophy_Optimization")

exercise_ids = range(len(df))
body_parts = df['BodyPart']
categories = df['Category']
equipment_types = df['Equipment']
sfr_values = df['Stimulus-to-Fatigue']
difficulty_ratings = df['Difficulty']

# Decision variables
x = model.addVars(len(df), lb=0, ub=1, vtype=GRB.CONTINUOUS, name="ExerciseProportion")

# Objective function: Maximize hypertrophy rating
model.setObjective(
    quicksum(df.loc[i, 'Hypertrophy Rating'] * x[i] for i in exercise_ids),
    GRB.MAXIMIZE
)

# Constraint 1: Total proportion must equal 1
model.addConstr(
    quicksum(x[i] for i in exercise_ids) == 1, 
    name="TotalProportionConstraint"
)

# Constraint 2: Limit the proportion per exercise to a maximum of 5%
for i in exercise_ids:
    model.addConstr(x[i] <= 0.05, name=f"MaxProportion_{i}")

# Constraint 3: Enforce minimum allocations for specific and general body parts
general_min_allocation = 0.025  # 2.5%
specific_min_allocations = {
    'Traps': 0.005,      # 0.5%
    'Neck': 0.005,       # 0.5%
    'Forearms': 0.005,   # 0.5%
    'Abdominals': 0.04   # 4%
}

for part in df['BodyPart'].unique():
    if part in specific_min_allocations:
        model.addConstr(
            quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= specific_min_allocations[part],
            f"{part}Min"
        )
    else:
        model.addConstr(
            quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= general_min_allocation,
            f"{part}GeneralMin"
        )

# Constraint 4: Leg muscles allocation ≥ 2.6 × upper body allocation
leg_muscles = ['Adductors', 'Abductors', 'Calves', 'Glutes', 'Hamstrings', 'Quadriceps']
upper_body_muscles = ['Chest', 'Lower Back', 'Middle Back', 'Biceps', 'Traps', 'Triceps', 
                      'Shoulders', 'Abdominals', 'Forearms', 'Neck', 'Lats']

model.addConstr(
    quicksum(x[i] for i in exercise_ids if body_parts[i] in leg_muscles) >= 
    2.6 * quicksum(x[i] for i in exercise_ids if body_parts[i] in upper_body_muscles),
    "LegUpperBodyRatio"
)

# Constraint 5: Biceps and triceps allocations balance with chest, lower back, and middle back
model.addConstr(
    quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Biceps', 'Triceps']) ==
    quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Chest', 'Lower Back', 'Middle Back']),
    "MuscleGroupBalance"
)

# Constraint 6: Stimulus-to-Fatigue Ratio (SFR) ≤ 0.55
model.addConstr(
    quicksum(sfr_values[i] * x[i] for i in exercise_ids) <= 0.55,
    "SFRConstraint"
)

# Constraint 7: Beginner ≥ 1.4 × intermediate ≥ advanced difficulty ratios
model.addConstr(
    quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Beginner') >= 
    1.4 * quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Intermediate'),
    "BeginnerIntermediateRatio"
)

model.addConstr(
    quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Intermediate') >= 
    1.1 * quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Advanced'),
    "IntermediateAdvancedRatio"
)

# Constraint 8: Strongman, Powerlifting, and Olympic Weightlifting allocations
model.addConstr(
    quicksum(x[i] for i in exercise_ids if categories[i] == 'Strongman') <= 0.08,
    "StrongmanMax"
)
model.addConstr(
    quicksum(x[i] for i in exercise_ids if categories[i] == 'Powerlifting') >= 0.09,
    "PowerliftingMin"
)
model.addConstr(
    quicksum(x[i] for i in exercise_ids if categories[i] == 'Olympic Weightlifting') >= 0.10,
    "OlympicWeightliftingMin"
)

# Constraint 9: At least 60% of exercises involve essential equipment
essential_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
model.addConstr(
    quicksum(x[i] for i in exercise_ids if equipment_types[i] in essential_equipment) >= 0.6,
    "EssentialEquipmentMin"
)

# Solve the model
model.optimize()

# Debug and display results
if model.Status == GRB.OPTIMAL:
    total_proportion = sum(x[i].x for i in exercise_ids)
    print(f"Total Proportion: {total_proportion:.4f}")
    print(f"The optimal solution: {model.ObjVal:.4f}")
    for i in exercise_ids:
        if x[i].x > 0:  # Display only non-zero proportions
            print(f"Exercise {df.loc[i, 'Exercise']}: Proportion = {x[i].x:.4f}")
else:
    print(f"Optimization status: {model.Status}")
    if model.Status == GRB.INFEASIBLE:
        print("Model is infeasible. Computing IIS...")
        model.computeIIS()
        model.write("infeasible_constraints.ilp")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2664 rows, 2637 columns and 20237 nonzeros
Model fingerprint: 0x07060e9c
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 2638 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 2637 columns, 15326 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.805000e+00   0.000000e+00      0s
      31    7.6726674e-01   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.00 seconds (0.01 work units)
Optimal objective  7.672667395e-01
Total Proportion: 1.0000
The optimal solution: 0.7673
Exercise Clean Deadlift: Proportion = 0.0500
Exercise Muscle Snatch: Proportion = 0.0500
Exercise Clean Shrug:

### Shadow Prices

In [133]:
#geting the shadow prices of the model 
# Loop through all constraints and print their shadow prices
for constr in model.getConstrs():
    print(f"Constraint {constr.ConstrName}: Shadow Price = {constr.Pi}")

Constraint TotalProportionConstraint: Shadow Price = 0.06283635645299192
Constraint MaxProportion_0: Shadow Price = 0.0
Constraint MaxProportion_1: Shadow Price = 0.0
Constraint MaxProportion_2: Shadow Price = 0.0
Constraint MaxProportion_3: Shadow Price = 0.0
Constraint MaxProportion_4: Shadow Price = 0.0
Constraint MaxProportion_5: Shadow Price = 0.0
Constraint MaxProportion_6: Shadow Price = 0.0
Constraint MaxProportion_7: Shadow Price = 0.0
Constraint MaxProportion_8: Shadow Price = 0.0
Constraint MaxProportion_9: Shadow Price = 0.0
Constraint MaxProportion_10: Shadow Price = 0.0
Constraint MaxProportion_11: Shadow Price = 0.0
Constraint MaxProportion_12: Shadow Price = 0.0
Constraint MaxProportion_13: Shadow Price = 0.0
Constraint MaxProportion_14: Shadow Price = 0.0
Constraint MaxProportion_15: Shadow Price = 0.0
Constraint MaxProportion_16: Shadow Price = 0.0
Constraint MaxProportion_17: Shadow Price = 0.0
Constraint MaxProportion_18: Shadow Price = 0.0
Constraint MaxProportion_

### 2. d) If the SFR requirement (i.e., constraint 5) were relaxed by 0.001, how much would the hypertrophy rating of the workout program improve by? Is this estimate valid?

In [134]:
from gurobipy import Model, GRB, quicksum

# Initialize the Gurobi model
model_s = Model("Hypertrophy SFR")

exercise_ids = range(len(df))
body_parts = df['BodyPart']
categories = df['Category']
equipment_types = df['Equipment']
sfr_values = df['Stimulus-to-Fatigue']
difficulty_ratings = df['Difficulty']

# Decision variables
x = model_s.addVars(len(df), lb=0, ub=1, vtype=GRB.CONTINUOUS, name="ExerciseProportion")

# Objective function: Maximize hypertrophy rating
model_s.setObjective(
    quicksum(df.loc[i, 'Hypertrophy Rating'] * x[i] for i in exercise_ids),
    GRB.MAXIMIZE
)

# Constraint 1: Total proportion must equal 1
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids) == 1, 
    name="TotalProportionConstraint"
)

# Constraint 2: Limit the proportion per exercise to a maximum of 5%
for i in exercise_ids:
    model_s.addConstr(x[i] <= 0.05, name=f"MaxProportion_{i}")

# Constraint 3: Enforce minimum allocations for specific and general body parts
general_min_allocation = 0.025  # 2.5%
specific_min_allocations = {
    'Traps': 0.005,      # 0.5%
    'Neck': 0.005,       # 0.5%
    'Forearms': 0.005,   # 0.5%
    'Abdominals': 0.04   # 4%
}

for part in df['BodyPart'].unique():
    if part in specific_min_allocations:
        model_s.addConstr(
            quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= specific_min_allocations[part],
            f"{part}Min"
        )
    else:
        model_s.addConstr(
            quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= general_min_allocation,
            f"{part}GeneralMin"
        )

# Constraint 4: Leg muscles allocation ≥ 2.6 × upper body allocation
leg_muscles = ['Adductors', 'Abductors', 'Calves', 'Glutes', 'Hamstrings', 'Quadriceps']
upper_body_muscles = ['Chest', 'Lower Back', 'Middle Back', 'Biceps', 'Traps', 'Triceps', 
                      'Shoulders', 'Abdominals', 'Forearms', 'Neck', 'Lats']

model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if body_parts[i] in leg_muscles) >= 
    2.6 * quicksum(x[i] for i in exercise_ids if body_parts[i] in upper_body_muscles),
    "LegUpperBodyRatio"
)

# Constraint 5: Biceps and triceps allocations balance with chest, lower back, and middle back
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Biceps', 'Triceps']) ==
    quicksum(x[i] for i in exercise_ids if body_parts[i] in ['Chest', 'Lower Back', 'Middle Back']),
    "MuscleGroupBalance"
)

# Constraint 6: Stimulus-to-Fatigue Ratio (SFR) ≤ 0.55
model_s.addConstr(
    quicksum(sfr_values[i] * x[i] for i in exercise_ids) <= 0.551,
    "SFRConstraint"
)

# Constraint 7: Beginner ≥ 1.4 × intermediate ≥ advanced difficulty ratios
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Beginner') >= 
    1.4 * quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Intermediate'),
    "BeginnerIntermediateRatio"
)

model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Intermediate') >= 
    1.1 * quicksum(x[i] for i in exercise_ids if difficulty_ratings[i] == 'Advanced'),
    "IntermediateAdvancedRatio"
)

# Constraint 8: Strongman, Powerlifting, and Olympic Weightlifting allocations
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if categories[i] == 'Strongman') <= 0.08,
    "StrongmanMax"
)
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if categories[i] == 'Powerlifting') >= 0.09,
    "PowerliftingMin"
)
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if categories[i] == 'Olympic Weightlifting') >= 0.10,
    "OlympicWeightliftingMin"
)

# Constraint 9: At least 60% of exercises involve essential equipment
essential_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
model_s.addConstr(
    quicksum(x[i] for i in exercise_ids if equipment_types[i] in essential_equipment) >= 0.6,
    "EssentialEquipmentMin"
)

# Solve the model
model_s.optimize()

# Debug and display results
if model_s.Status == GRB.OPTIMAL:
    total_proportion = sum(x[i].x for i in exercise_ids)
    print(f"Total Proportion: {total_proportion:.4f}")
    print(f"The optimal solution: {model_s.ObjVal:.4f}")
    for i in exercise_ids:
        if x[i].x > 0:  # Display only non-zero proportions
            print(f"Exercise {df.loc[i, 'Exercise']}: Proportion = {x[i].x:.4f}")
else:
    print(f"Optimization status: {model_s.Status}")
    if model_s.Status == GRB.INFEASIBLE:
        print("Model is infeasible. Computing IIS...")
        model_s.computeIIS()
        model_s.write("infeasible_constraints.ilp")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2664 rows, 2637 columns and 20237 nonzeros
Model fingerprint: 0xe34a88f9
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 2638 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 2637 columns, 15326 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.805000e+00   0.000000e+00      0s
      31    7.6864373e-01   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.00 seconds (0.01 work units)
Optimal objective  7.686437338e-01
Total Proportion: 1.0000
The optimal solution: 0.7686
Exercise Clean Deadlift: Proportion = 0.0500
Exercise Muscle Snatch: Proportion = 0.0500
Exercise Clean Shrug:

###  Part (f): Including Barbell Back Squats

In [135]:
exercise_name = "Barbell Back Squats"
exercise_index = df[df['Exercise'] == exercise_name].index[0]

variable = x[exercise_index]

if model.status == GRB.OPTIMAL:
    print(f"Sensitivity Information for '{exercise_name}':")
    print(f"  Current Hypertrophy Rating: {df['Hypertrophy Rating'][exercise_index]:.4f}")
    print(f"  Increase Limit (SAObjUp): {variable.SAObjUp:.4f}")
    print(f"  Decrease Limit (SAObjLow): {variable.SAObjLow:.4f}")
else:
    print("Model is not optimized.")

Sensitivity Information for 'Barbell Back Squats':
  Current Hypertrophy Rating: 0.6783
  Increase Limit (SAObjUp): 1.1491
  Decrease Limit (SAObjLow): -inf


### 2. h) Suppose that all of the common constraints are removed except {1, 2, 8}. What is the optimal hypertrophy rating, and why is it higher than in the original solution?

In [136]:
# Create a new optimization model
model_p = gb.Model("Primal Model")

# Decision variables
num_exercises = len(df)
x = model_p.addVars(num_exercises, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="ExerciseProportion")

# Objective function: Maximizes the total hypertrophy rating across all exercises based on their proportions.
model_p.setObjective(
        quicksum(df.loc[i, 'Hypertrophy Rating'] * x[i] for i in range(num_exercises)),
        GRB.MAXIMIZE
    )

# Adding the constraints
## 1. Limit the proportion per exercise to a maximum of 5%.
for i in range(num_exercises):
    model_p.addConstr(x[i] <= 0.05, name=f"MaxProportion_{i}")

# 2. Enforce minimum allocations for specific and general body parts.
general_min_allocation = 0.025  # 2.5%
specific_min_allocations = {
    'Traps': 0.005,      # 0.5%
    'Neck': 0.005,       # 0.5%
    'Forearms': 0.005,   # 0.5%
    'Abdominals': 0.04   # 4%
}

# General and specific allocation constraints for body parts
unique_body_parts = df['BodyPart'].unique()
for part in unique_body_parts:
    if part in specific_min_allocations:
        model_p.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= specific_min_allocations[part],
            f"{part}Min"
        )
    else:
        model_p.addConstr(
            gb.quicksum(x[i] for i in exercise_ids if body_parts[i] == part) >= general_min_allocation,
            f"{part}GeneralMin"
        )

# 8. Ensure at least 60% of exercises involve essential equipment types.
essential_equipment = ['Barbell', 'Dumbbell', 'Machine', 'Cable', 'E-Z Curl Bar', 'Bands']
equipment_constraint = model_p.addConstr(
    gb.quicksum(x[i] for i in exercise_ids if equipment_types[i] in essential_equipment) >= 0.6,
    "EssentialEquipmentMin"
)

# Ensure total proportion is 1
model_p.addConstr(gb.quicksum(x[i] for i in range(len(df))) == 1, "Total_Proportion")

# Solve the optimization model to find the optimal proportions for each exercise while adhering to all constraints.
# The model successfully converges to an optimal solution with an objective value of 0.7672, maximizing hypertrophy.
model_p.optimize()

if model_p.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    model_p.computeIIS()
    model_p.write("infeasible_constraints.ilp")
    
# Check the optimization status
if model_p.status == GRB.OPTIMAL:
    print("The optimal solution: ", model_p.objVal)
    # Optionally, display the decision variables
    for i in exercise_ids:
        if x[i].x > 0:
            print(f"Exercise {df.loc[i, 'Exercise']}: Proportion = {x[i].x:.4f}")
elif model_p.status == GRB.INFEASIBLE:
    print("The model is infeasible.")
elif model_p.status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print("Optimization was stopped with status", model_p.status)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2656 rows, 2637 columns and 9217 nonzeros
Model fingerprint: 0xcc26d03a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 2637 rows and 0 columns
Presolve time: 0.00s
Presolved: 19 rows, 2637 columns, 6580 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   1.305000e+00   0.000000e+00      0s
      14    8.5703576e-01   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.570357586e-01
The optimal solution:  0.8570357585549997
Exercise Behind-the-head push-press: Proportion = 0.0500
Exercise Board bench press: Proportion = 0.0500
Exercise Barbell gl

### Part(i): Formulating and Solving the Dual Problem

In [137]:
from gurobipy import Model, GRB, quicksum

def solve_dual(df):
    """
    Solve the dual problem for the primal optimization.
    """
    # Create the dual model
    dual_model = Model("Dual_Problem")

    # Primal data
    num_exercises = len(df)
    muscle_groups = {"Traps", "Neck", "Forearms", "Abdominals"}
    equipment_types = ["Barbell", "Dumbbell", "Machine", "Cable", "E-Z Curl Bar", "Bands"]

    # Define muscle group thresholds from the primal
    muscle_thresholds = {
        "Traps": 0.005,
        "Neck": 0.005,
        "Forearms": 0.005,
        "Abdominals": 0.04
    }
    default_body_part_weight = 0.025

    # Dual variables
    mu = dual_model.addVars(num_exercises, lb=0, vtype=GRB.CONTINUOUS, name="mu")
    nu = dual_model.addVars(muscle_groups, lb=0, vtype=GRB.CONTINUOUS, name="nu")
    lambda_var = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="lambda")
    sigma = dual_model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="sigma")

    # Objective function: Minimize
    dual_model.setObjective(
        quicksum(0.05 * mu[i] for i in range(num_exercises)) +
        quicksum(nu[g] * muscle_thresholds[g] for g in muscle_groups) +
        lambda_var * 0.60 +
        sigma,
        GRB.MINIMIZE
    )

    # Dual constraints
    for i in range(num_exercises):
        body_part = df['BodyPart'].iloc[i]
        equipment = df['Equipment'].iloc[i]
        hypertrophy_rating = df['Hypertrophy Rating'].iloc[i]

        dual_model.addConstr(
            mu[i] +
            (nu[body_part] if body_part in muscle_groups else 0) +
            (lambda_var if equipment in equipment_types else 0) +
            sigma >= hypertrophy_rating,
            name=f"Dual_Constraint_{i}"
        )

    # Optimize the dual problem
    dual_model.optimize()

    # Output results
    if dual_model.status == GRB.OPTIMAL:
        print("Optimization completed successfully.")
        print(f"Optimal Dual Objective Value: {dual_model.objVal:.4f}")

        # Dual Variables
        print("\nDual Variables:")
        for v in dual_model.getVars():
            print(f"{v.varName}: {v.x:.4f}")

        # Constraint Slackness
        print("\nConstraint Slackness:")
        for constr in dual_model.getConstrs():
            print(f"{constr.constrName}: Slack = {constr.slack:.4f}")

    return dual_model.objVal

# Run the dual problem solver
dual_score = solve_dual(df)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2637 rows, 2643 columns and 7283 nonzeros
Model fingerprint: 0x14231c71
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-03, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+00]
Presolve removed 696 rows and 703 columns
Presolve time: 0.00s
Presolved: 1941 rows, 1940 columns, 5043 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.101312e+03   0.000000e+00      0s
      23    8.3749717e-01   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.00 seconds (0.00 work units)
Optimal objective  8.374971723e-01
Optimization completed successfully.
Optimal Dual Objective Value: 0.8375

Dual Variables:
mu[0]: 0.0000
mu[1]: 0.0000
mu[2]: 0.0000
mu[3]: 0.0000
mu[4]: 0.0000
mu